# Stimulus preprocessing

In this notebook, we will read the annotations for each file and create a dataset for all the images. Then we will go on to create different versions of our dataset as below.



In [1]:
from __future__ import division

In [2]:
from packages import *
%matplotlib inline

/home/kaandonbekci/src/modules/utils.py:2: DeprecationWarning: Python2 support is deprecated and will be removed in a future release. Consider switching to Python3.
  import os, nilearn.plotting, nibabel
/home/kaandonbekci/.local/lib/python2.7/site-packages/nilearn/plotting/js_plotting_utils.py:14: ImportWarning: Not importing directory 'html': missing __init__.py
  from html import escape  # Unavailable in Py2
/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binar

In [3]:
import json
import random
import math

In [4]:
import tensorflow.keras.backend as K
K.set_floatx('float32')
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
from get_activations import get_activations

In [6]:
subject = 1

In [7]:
run = True
build = False
predict = True
save = False
load = True
check = False

In [8]:
export_precision = np.float32

In [9]:
if load and not build:
    stim_labels = load_pickle('stimulus-labels.p')
    print('stimulus labels loaded, length: {}'.format(len(stim_labels)))

stimulus labels loaded, length: 4916


In [10]:
def batchify(l, batch_size):
    batches = []
    for i in xrange(0, len(l), batch_size):
        batches.append(l[i:i+batch_size])
    return batches

In [11]:
def preprocess_image(f, target_size, batch_dim_is_front=True):
    img = stim_labels[f]
    f_type = img['type']
    f_path = os.path.join(IMAGES_DIR[img['type']], f)
    original = load_img(f_path, target_size=target_size)
    img_arr = img_to_array(original)
    processed_img_arr = vgg16.preprocess_input(np.expand_dims(img_arr, 0))
    return processed_img_arr

In [12]:
def get_vgg_predictions():
    vgg_predictions = {}
    i = 0
    for f in stim_labels:
        vgg_predictions[f] = vgg.predict(preprocess_image(f, (224,224)))
        i += 1
        if i % 100 == 0:
            print i ,
    return vgg_predictions

In [13]:
if run and predict:
    vgg = vgg16.VGG16(weights = 'imagenet')

In [29]:
def create_stim_dicts():
    labels_dir = os.path.join(STIM_DIR, 'Image_Labels/')
    imagenet_labels = os.path.join(labels_dir, 'imagenet_final_labels.txt')
    output = {}
    prefix_labels = {}
    with open(imagenet_labels) as f:  
        for line in f:
            x = line.strip().split(' ', 1)
            labels = x[1].split(', ')
            filename = x[0]
            prefix_labels[filename] = {'labels': labels, 'type': 'imagenet'}
    
    imagenet_dir = os.path.join(STIM_DIR, 'Scene_Stimuli', 'Presented_Stimuli/', 'ImageNet/')
    for x in os.listdir(imagenet_dir):
        if x.startswith('._') or not x.endswith('.JPEG'):
            continue
        prefix = x.split('_')[0]
        output[x] = prefix_labels[prefix]
    print '{} imagenet files'.format(len(output))
    cat_lookup = {}
    with open(os.path.join(MISC_DIR, 'image_info_test2014.json')) as json_data:
        cats = json.load(json_data)
    for cat in cats['categories']:
        cat_lookup[cat['id']] = cat['name']
    with open(os.path.join(labels_dir, 'coco_final_annotations.pkl'), 'rb') as f:
        coco_labels = pickle.load(f)
        for key in coco_labels:
            filename =  'COCO_train2014_{}.jpg'.format(('%012.1d' % key))
            all_labels = set()
            for entry in coco_labels[key]:
                all_labels.add(entry['category_id'])
            labels = []
            for l in all_labels:
                labels.append(cat_lookup[l])
            output[filename] = {'labels': labels, 'type': 'coco'}
        print '{} coco files'.format(len(coco_labels))
    
    scene_dir = os.path.join(STIM_DIR, 'Scene_Stimuli', 'Presented_Stimuli/', 'Scene/')
    count = 0
    for x in os.listdir(scene_dir):      
        if x.startswith('._') or (not x.endswith('.jpg') and not x.endswith('.jpeg')):
            continue
        label = ''.join([ i for i in x.split('.')[0] if not i.isdigit()])
        output[x] = {'labels': [label], 'type': 'scenes'}
        count+=1
    print '{} scene files'.format(count)
    return output

In [30]:
if build:
    stim_data = create_stim_dicts()

In [31]:
if save and build:
    save_pickle(stim_data, 'stimulus-labels')

#### Let's make sure that everyhing is in order with our final dictionary.

In [17]:
if check:
    for i in xrange(100000):
        key = random.choice(output.keys())
        assert key.endswith('.jpg') or key.endswith('.JPEG') or  key.endswith('.jpeg'), 'Key: {}'.format(key)
        x = output[key]
        assert type(x) is dict
        assert type(x['labels']) is list
        assert type(x['type']) is str

# Building versions of the stimulus dataset

## Different datasets we will consider are: 

##### (a) VGG16 outputs (100d vector) for each stimuli
##### (b) one-hot VGG16 outputs for each stimuli
##### (c) stimuli images resized and preprocessed


### (a) VGG16 outputs

In [18]:
if run and predict:
    vgg_preds = get_vgg_predictions()

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800

KeyboardInterrupt: 

In [ ]:
if save and predict and not load:
    save_pickle(vgg_preds, 'vgg-preds')

### (b) one-hot VGG16 outputs

In [ ]:
if load and not predict:
    vgg_preds = load_pickle('vgg-preds.p')

In [ ]:
x = random.choice(vgg_preds.keys())
print x
plt.imshow(preprocess_image(x, (224, 224))[0,])
vgg16.decode_predictions(vgg_preds[x])[0][0:3]

In [ ]:
assert one_hot_vgg_preds[x] == np.argmax(vgg_preds[x])

In [ ]:
one_hot_vgg_preds = {}
for f in stim_labels:
    y = np.argmax(vgg_preds[f][0,])
    one_hot_vgg_preds[f] = y

In [ ]:
if save:
    save_pickle(one_hot_vgg_preds, 'one_hot-vgg-preds')

### (c) stimulus image processed, resized and flattened

resized to (50, 50, 3) and processed with the vgg16 preprocess tool

In [19]:
target_size = (50, 50)

In [20]:
stim_processed_resized_flat= {}
i = 0
for f in stim_labels:
    img = stim_labels[f]
    f_path = os.path.join(IMAGES_DIR[img['type']], f)
    original = load_img(f_path, target_size=target_size)
    img_arr = img_to_array(original)
    processed_img_arr = vgg16.preprocess_input(np.expand_dims(img_arr, 0))
    stim_processed_resized_flat[f] = processed_img_arr[0].flatten()
    i+=1 
    if i%100 == 0:
        print '*' ,

* * * * * * * * * * * * * * * * *

KeyboardInterrupt: 

In [ ]:
if save:
    save_pickle(stim_processed_resized_flat, 'stimulus-processed-resized-flat')

### (d) VGG16 layer activations

extract layer activations from VGG model when ran with the stimulus

In [19]:
subject = 1
session = 1

In [14]:
sample_num = 1000

In [15]:
def generate_random_sample_indices(shape):
    channels = shape[-1]
    sample_per_channel = int(math.ceil(sample_num/channels))
    dim = shape[1]
    indexes = []
    for channel in xrange(channels):
        channel_samples = []
        for _ in range(sample_per_channel):
            ix = np.where(arrays[dim].reshape(dim, dim) == np.random.choice(arrays[dim]))
            channel_samples.append((ix[0][0], ix[1][0]))
        indexes.append(channel_samples)
    return np.array(indexes)

In [16]:
layers = [layer.name for layer in vgg.layers if 'conv' in layer.name and 'block1' not in layer.name and 'block2' not in layer.name]
a = get_activations(vgg, preprocess_image('childsroom2.jpg', (224, 224)), K, layers)
layers = {}
for key in a.keys():
    layers[key] = a[key].shape

In [17]:
layers

{'block3_conv1': (1, 56, 56, 256),
 'block3_conv2': (1, 56, 56, 256),
 'block3_conv3': (1, 56, 56, 256),
 'block4_conv1': (1, 28, 28, 512),
 'block4_conv2': (1, 28, 28, 512),
 'block4_conv3': (1, 28, 28, 512),
 'block5_conv1': (1, 14, 14, 512),
 'block5_conv2': (1, 14, 14, 512),
 'block5_conv3': (1, 14, 14, 512)}

In [18]:
vgg_layer_activations_link = {}
arrays = {14: np.arange(14**2), 28: np.arange(28**2), 56: np.arange(56**2)}
vgg_activation_masks = {}
vgg_activations = {}
for layer in layers:
    vgg_activation_masks[layer] = generate_random_sample_indices(layers[layer])
    vgg_activations[layer]= []
i = 0
for f in stim_labels:
    vgg_layer_activations_link[f] = i
    i+=1
    a = get_activations(vgg, preprocess_image(f, (224, 224)), K, layers.keys())
    for layer in a:
        layer_samples = []
        for channel in xrange(layers[layer][-1]):
            for sample in xrange(vgg_activation_masks[layer].shape[1]):
                layer_samples.append(a[layer][0, vgg_activation_masks[layer][channel, sample, 0], vgg_activation_masks[layer][channel, sample, 1], channel])
        vgg_activations[layer].append(np.array(layer_samples).astype(export_precision))
    if i%10 == 0:
        print '*',
    if i%100 == 0:
        print '', i*100/5000 ,

* * * * * * * * * *  2.0 * * * * * * * * * *  4.0 * * * * * * * * * *  6.0 * * * * * * * * * *  8.0 * * * * * * * * * *  10.0 * * * * * * * * * *  12.0 * * * * * * * * * *  14.0 * * * * * * * * * *  16.0 * * * * * * * * * *  18.0 * * * * * * * * * *  20.0 * * *

KeyboardInterrupt: 